In [43]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Define the training data
train_data = [
    ("Book a table at 7pm at the Italian restaurant.", "book_table"),
    ("I want to order a pizza for delivery.", "order_food"),
    ("What time does the store open tomorrow?", "store_hours"),
    ("Can you help me find a hotel near the airport?", "find_hotel"),
]

# Define the mapping between labels and integers
label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Convert the training data labels to integers using the label_map
# A tensor is a multi-dimensional array that looks like a numpy array, it's used for neural networks
labels = torch.tensor([label_map[data[1]] for data in train_data])

# Load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_map))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the training data and convert to tensors
inputs = tokenizer.batch_encode_plus([data[0] for data in train_data], padding=True, truncation=True, return_tensors="pt")

# Fine-tune the model on the training data
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(10):
    outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Test the model
text = "What time does the Italian restaurant open tomorrow?"

# inputs = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors="pt")
# outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
# predicted_label = torch.argmax(outputs.logits).item()
# predicted_intent = [k for k, v in label_map.items() if v == predicted_label][0]
# print(predicted_intent)

def predict_intent(text):
    inputs = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors="pt")
    outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    predicted_label = torch.argmax(outputs.logits).item()
    predicted_intent = [k for k, v in label_map.items() if v == predicted_label][0]
    
    return predicted_intent

predict_intent(text)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1, Loss: 1.388498067855835
Epoch 2, Loss: 1.2602953910827637
Epoch 3, Loss: 1.193776249885559
Epoch 4, Loss: 1.1271312236785889
Epoch 5, Loss: 1.0456219911575317
Epoch 6, Loss: 0.9578683972358704
Epoch 7, Loss: 0.9625304341316223
Epoch 8, Loss: 0.8976312875747681
Epoch 9, Loss: 0.8667942881584167
Epoch 10, Loss: 0.8594805002212524


'store_hours'

In [44]:
import gradio as gr
import time

# load the pre-trained intent analysis model
# nlp = spacy.load("en_trf_bertbaseuncased_lg")

response_map = {
    "book_table": ["Your table has been booked.", "Reservation confirmed."],
    "order_food": ["Your pizza is on the way.", "Delivery confirmed."],
    "store_hours": ["We are open from 9am to 10pm.", "Our hours are 9am-10pm, 7 days a week."],
    "find_hotel": ["There are several hotels near the airport.", "Here are some hotels near the airport: ..."]
}


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        predicted_intent = predict_intent(user_message)
        response = random.choice(response_map[predicted_intent])
        history[-1][1] = response
        time.sleep(1)
        return history



    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


a

#### Source:

 https://www.section.io/engineering-education/intent-classification-with-rasa-and-spacy/